In [5]:
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
import numpy as np



face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

emotion_model = load_model('emotion_cnn_48x48.keras')

EMOTIONS = ['Angry', 'Fear', 'Happy', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)
id = 0

while cap.isOpened(): 
    ret,frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=6)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract face ROI
        face_gray = gray[y:y + h, x:x + w]

        # Preprocess for emotion model
        face_resized = cv2.resize(face_gray, (48, 48))
        face_normalized = face_resized / 255.0
        face_input = face_normalized.reshape(1, 48, 48, 1)

        # Predict emotion
        preds = emotion_model.predict(face_input, verbose=0)
        emotion_idx = np.argmax(preds)
        emotion_label = EMOTIONS[emotion_idx]

        # Display emotion
        cv2.putText(
            frame,
            emotion_label,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (255, 0, 0),
            2
        )
    
    cv2.imshow('WebCam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()